# Data Analysis with Visual Analytics
**Combining some basic analytics with visualization**

NOTE: goal is to show benifit of easy RAPIDS framework compatability, and speed of which can do iterations on large(ish) dataset

## Overview and Requirements
Super short version of intro notebook and restate requirments

## Import

## Load Cleaned Data / Check

## Run some analysis -> ? = profit?
Mix of cuML / cuspatial analysis to then visualize

## Intro to cuML / cuSpatial Other Libraries
Overview of RAPIDS libraies and uses

## Iteration 1
cuxfilter results, tweak parameters

## Iteration 2
cxfilter results, tweak parameters

## Summary of interesting analytics results 
Does not have to be significant but noteable